# Билет 23

# Задание 2

In [143]:
import nltk
import pymorphy2
from collections import defaultdict

In [144]:
def read_text(path: str) -> str:
    with open(path, encoding='cp1251') as f:
        return f.read()

In [145]:
text = read_text('data/task_2/AnnaKarenina_.txt')
words = [w.lower() for s in nltk.sent_tokenize(text) for w in nltk.word_tokenize(s)]
len(words)

344931

In [146]:
words_truncated = [w for w in words if 3 <= len(w) <= 7]  # примерные границы: Анн - Аннами_
len(words_truncated)

150124

In [147]:
# для большего ускорения
words_truncated = [w for w in words if w.startswith('анн')]  # неизменяемая часть
len(words_truncated)

845

In [148]:
target_name = 'анна'
lemmatizer = pymorphy2.MorphAnalyzer()

name_forms = defaultdict(list)
for w in words_truncated:
    lemma = lemmatizer.parse(w)[0]
    if lemma.normal_form == target_name and lemma.tag.POS == 'NOUN':
        tag = lemma.tag
        name_forms[tag.case].append(lemma)

print(len(name_forms))
name_forms

4


defaultdict(list,
            {'nomn': [Parse(word='анна', tag=OpencorporaTag('NOUN,anim,femn,Name sing,nomn'), normal_form='анна', score=1.0, methods_stack=((DictionaryAnalyzer(), 'анна', 69, 0),)),
              Parse(word='анна', tag=OpencorporaTag('NOUN,anim,femn,Name sing,nomn'), normal_form='анна', score=1.0, methods_stack=((DictionaryAnalyzer(), 'анна', 69, 0),)),
              Parse(word='анна', tag=OpencorporaTag('NOUN,anim,femn,Name sing,nomn'), normal_form='анна', score=1.0, methods_stack=((DictionaryAnalyzer(), 'анна', 69, 0),)),
              Parse(word='анна', tag=OpencorporaTag('NOUN,anim,femn,Name sing,nomn'), normal_form='анна', score=1.0, methods_stack=((DictionaryAnalyzer(), 'анна', 69, 0),)),
              Parse(word='анна', tag=OpencorporaTag('NOUN,anim,femn,Name sing,nomn'), normal_form='анна', score=1.0, methods_stack=((DictionaryAnalyzer(), 'анна', 69, 0),)),
              Parse(word='анна', tag=OpencorporaTag('NOUN,anim,femn,Name sing,nomn'), normal_form='анна'

In [149]:
{k: len(v) for k, v in name_forms.items()}

{'nomn': 497, 'gent': 121, 'accs': 65, 'ablt': 39}

In [150]:
# здесь закончил эксперимент
{v[0].word: len(v) for k, v in name_forms.items()}

{'анна': 497, 'анны': 121, 'анну': 65, 'анной': 39}

In [151]:
# Дополнительно:
# не совсем корректное задание, т.к. форма слова зависит еще и от числа
target_name = 'анна'
lemmatizer = pymorphy2.MorphAnalyzer()

name_forms = defaultdict(list)
for w in words_truncated:
    lemma = lemmatizer.parse(w)[0]
    if lemma.normal_form == target_name and lemma.tag.POS == 'NOUN':
        tag = lemma.tag
        name_forms[(tag.number, tag.case)].append(lemma)

len(name_forms)

5

In [152]:
{(*k, v[0].word): len(v) for k, v in name_forms.items()}

{('sing', 'nomn', 'анна'): 497,
 ('sing', 'gent', 'анны'): 120,
 ('sing', 'accs', 'анну'): 65,
 ('sing', 'ablt', 'анной'): 39,
 ('plur', 'gent', 'анн'): 1}

In [160]:
# финальное решение!

def for_name(name, text):
    lemmatizer = pymorphy2.MorphAnalyzer()

    name = lemmatizer.parse(name)[0].normal_form
    print(f'Целевое имя: {name}')

    words = [w.lower() for s in nltk.sent_tokenize(text) for w in nltk.word_tokenize(s)]
    words_truncated = [w for w in words if len(w) <= len(name) + 3 and name.startswith(name[:2])]
    print(f'Слов для анализа: {len(words_truncated)}')

    name_forms = defaultdict(list)
    for w in words_truncated:
        lemma = lemmatizer.parse(w)[0]
        if lemma.normal_form == name and lemma.tag.POS == 'NOUN':
            tag = lemma.tag
            name_forms[(tag.number, tag.case)].append(lemma)

    return {(*k, v[0].word): len(v) for k, v in name_forms.items()}

In [161]:
text = read_text('data/task_2/AnnaKarenina_.txt')

In [162]:
for_name('анна', text)

Целевое имя: анна
Слов для анализа: 291144


{('sing', 'nomn', 'анна'): 497,
 ('sing', 'gent', 'анны'): 120,
 ('sing', 'accs', 'анну'): 65,
 ('sing', 'ablt', 'анной'): 39,
 ('plur', 'gent', 'анн'): 1}

In [163]:
for_name('алексей', text)

Целевое имя: алексей
Слов для анализа: 332312


{('sing', 'gent', 'алексея'): 130,
 ('sing', 'nomn', 'алексей'): 429,
 ('sing', 'datv', 'алексею'): 46,
 ('sing', 'ablt', 'алексеем'): 23,
 ('sing', 'loct', 'алексее'): 6,
 ('plur', 'nomn', 'алексеи'): 1}

In [164]:
for_name('дарья', text)

Целевое имя: дарья
Слов для анализа: 310319


{('sing', 'datv', 'дарье'): 31,
 ('sing', 'nomn', 'дарья'): 146,
 ('sing', 'gent', 'дарьи'): 22,
 ('sing', 'accs', 'дарью'): 11,
 ('sing', 'ablt', 'дарьей'): 6}

# Задание 3

In [157]:
import dask.dataframe as dd

In [158]:
accounts: dd.DataFrame = dd.read_csv(
    'data/task_3/data/accounts.*.csv',
    dtype={'id': int, 'names': object, 'amount': int}
)
accounts.head()

,id,names,amount
0,187,Ingrid,457
1,346,Ursula,747
2,350,Dan,1175
3,464,Zelda,-571
4,431,Wendy,4534


In [159]:
accounts_mul = accounts[accounts['amount'] % 3 == 0]
id_ = accounts_mul.groupby('id')['amount'].count().idxmax().compute()
id_

366